# Atomic Simulation with Python

## 1. Simulating the energetics for oxygen diffusion on platinum.

In [ ]:
# Calculate O diffusion Energy on a Platinum Surface
# Let's load the python packages we will need
from ase.build import fcc111, add_adsorbate
from ase.constraints import FixAtoms
from ase.calculators.emt import EMT
from ase.optimize import QuasiNewton
from ase.visualize import view

# 4x4-Pt(111) surface with 4 layers and an
# O atom adsorbed in a fcc-hollow site:
slab = fcc111('Pt', size=(4, 4, 4))
slab.center(axis=2, vacuum=4.0)
add_adsorbate(slab, 'O', 1.25, 'fcc')

# Fix second and third layers:
mask = [atom.tag > 1 for atom in slab]
#print(mask)
slab.set_constraint(FixAtoms(mask=mask))
# Use EMT potential:
slab.set_calculator(EMT())

# We are going to calculate the structure of the Initial state by minimizing
#the forces on atoms through numerical optimization :
qn = QuasiNewton(slab, trajectory='initial.traj')
qn.run(fmax=0.01)
view(slab)
print(slab)



In [ ]:
# 4x4-Pt(111) surface with 4 layers and an
# O atom adsorbed on the other side of the surface:
slab = fcc111('Pt', size=(4, 4, 4))
slab.center(axis=2, vacuum=4.0)
add_adsorbate(slab, 'O', 1.25, position=(10,6))


# Fix second and third layers:
mask = [atom.tag > 1 for atom in slab]
#print(mask)
slab.set_constraint(FixAtoms(mask=mask))
# Use EMT potential:
slab.set_calculator(EMT())

# Final state:
qn = QuasiNewton(slab, trajectory='final.traj')
qn.run(fmax=0.01)
view(slab)
print(slab)

In [ ]:
# Next we are going to take our initial and final structures and do 
#a simulation that will calculate the energy to move between them
from ase.io import read
from ase.constraints import FixAtoms
from ase.calculators.emt import EMT
from ase.neb import NEB
from ase.optimize import BFGS

initial = read('initial.traj')
final = read('final.traj')

constraint = FixAtoms(mask=[atom.tag > 1 for atom in initial])

images = [initial]
for i in range(9):
    image = initial.copy()
    image.set_calculator(EMT())
    image.set_constraint(constraint)
    images.append(image)

images.append(final)

neb = NEB(images)
neb.interpolate()
qn = BFGS(neb, trajectory='O_diffusion.traj')
qn.run(fmax=0.01)

In [ ]:
#The code below will take our simulation results and allow us to 
# visualize them
import matplotlib.pyplot as plt
from ase.neb import NEBTools
from ase.io import read

images = read('O_diffusion.traj@-11:')

nebtools = NEBTools(images)

# Get the calculated barriers and the energy change of the reaction.
Ef, dE = nebtools.get_barrier()

# Get the barriers without any interpolation between highest images.
Ef, dE = nebtools.get_barrier(fit=False)

# Get the actual maximum force at this point in the simulation.
max_force = nebtools.get_fmax()

# Create a figure.
fig = nebtools.plot_band()
fig.savefig('diffusion-barrier.png')

view(images)

## 2. Simulating the energetics for dioxygen dissociation and diffusion on platinum.

In [ ]:
# 4x4-Pt(111) surface with 4 layers and an
# O2 molecule adsorbed in a fcc-hollow site:
slab = fcc111('Pt', size=(4, 4, 4))
slab.center(axis=2, vacuum=4.0)
add_adsorbate(slab, 'O', 1.25, 'fcc')
add_adsorbate(slab, 'O', 2.45, 'fcc')

# Fix second and third layers:
mask = [atom.tag > 1 for atom in slab]
#print(mask)
slab.set_constraint(FixAtoms(mask=mask))
# Use EMT potential:
slab.set_calculator(EMT())

# Initial state:
qn = QuasiNewton(slab, trajectory='initial.traj')
qn.run(fmax=0.01)
view(slab)
print(slab)

In [ ]:
# 4x4-Pt(111) surface with 4 layers and an
# 2 O atoms adsorbed:
slab = fcc111('Pt', size=(4, 4, 4))
slab.center(axis=2, vacuum=4.0)
add_adsorbate(slab, 'O', 1.25, 'fcc')
add_adsorbate(slab, 'O', 1.25, position=(10,6))




# Fix second and third layers:
mask = [atom.tag > 1 for atom in slab]
#print(mask)
slab.set_constraint(FixAtoms(mask=mask))
# Use EMT potential:
slab.set_calculator(EMT())

# Initial state:
qn = QuasiNewton(slab, trajectory='final.traj')
qn.run(fmax=0.01)
view(slab)
print(slab)

In [ ]:
initial = read('initial.traj')
final = read('final.traj')

constraint = FixAtoms(mask=[atom.tag > 1 for atom in initial])

images = [initial]
for i in range(11):
    image = initial.copy()
    image.set_calculator(EMT())
    image.set_constraint(constraint)
    images.append(image)

images.append(final)

neb = NEB(images)
neb.interpolate()
qn = BFGS(neb, trajectory='O2_dissociation.traj')
qn.run(fmax=0.1)

In [ ]:
images = read('O2_dissociation.traj@-13:')

nebtools = NEBTools(images)

# Get the calculated barrier and the energy change of the reaction.
Ef, dE = nebtools.get_barrier()

# Get the barrier without any interpolation between highest images.
Ef, dE = nebtools.get_barrier(fit=False)

# Get the actual maximum force at this point in the simulation.
max_force = nebtools.get_fmax()

# Create a figure like that coming from ASE-GUI.
fig = nebtools.plot_band()
fig.savefig('diffusion-barrier.png')
# To view movie with ase gui $ase gui neb.traj@-5:
# Then use NEB tools

view(images)

## Molecular Dynamics Simulation of dioxygen on platinum

In [ ]:
from ase.lattice.cubic import FaceCenteredCubic
from ase.md.langevin import Langevin
from ase.io.trajectory import Trajectory
from ase import units
from ase.md.velocitydistribution import (MaxwellBoltzmannDistribution, Stationary)


T = 200  # Kelvin
slab = read('initial.traj')
slab.set_constraint()

slab.set_calculator(EMT())
MaxwellBoltzmannDistribution(slab, temperature_K=100)
Stationary(slab)  # zero linear momentum

# We want to run MD with constant energy using the Langevin algorithm
# with a time step of 5 fs, the temperature T and the friction
# coefficient to 0.02 atomic units.
dyn = Langevin(slab, 5 * units.fs, T * units.kB, 0.002)


def printenergy(a=slab):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin))


dyn.attach(printenergy, interval=10)

# We also want to save the positions of all atoms after every 100th time step.
traj = Trajectory('moldyn.traj', 'w', slab)
dyn.attach(traj.write, interval=1)

# Now run the dynamics
printenergy()
dyn.run(1000)

In [ ]:
images = read('moldyn.traj',':')
view(images)

## 3. CO Oxidation

In [ ]:
# Generate structure with O2 and CO
slab = fcc111('Pt', size=(4, 4, 4))
slab.center(axis=2, vacuum=4.0)
add_adsorbate(slab, 'O', 1.25, 'fcc')
add_adsorbate(slab, 'O', 2.45, 'fcc')
add_adsorbate(slab, 'C', 1.25, position=(9.5,5.5))
add_adsorbate(slab, 'O', 2.25, position=(9.5,5.5))




# Fix second and third layers:
mask = [atom.tag > 1 for atom in slab]
#print(mask)
slab.set_constraint(FixAtoms(mask=mask))
# Use EMT potential:
slab.set_calculator(EMT())

# Initial state:
qn = QuasiNewton(slab, trajectory='initial.traj')
qn.run(fmax=0.01)
view(slab)
print(slab)

In [ ]:
# Run MD
T = 250  # Kelvin
slab = read('initial.traj')
slab.set_constraint()

slab.set_calculator(EMT())
MaxwellBoltzmannDistribution(slab, temperature_K=200)
Stationary(slab)  # zero linear momentum


# We want to run MD with constant energy using the Langevin algorithm
# with a time step of 5 fs, the temperature T and the friction
# coefficient to 0.02 atomic units.
dyn = Langevin(slab, 5 * units.fs, T * units.kB, 0.002)


def printenergy(a=slab):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin))


dyn.attach(printenergy, interval=10)

# We also want to save the positions of all atoms after every 100th time step.
traj = Trajectory('moldyn.traj', 'w', slab)
dyn.attach(traj.write, interval=1)

# Now run the dynamics
printenergy()
dyn.run(1000)

In [ ]:
images = read('moldyn.traj',':')
view(images)